In [3]:
import pandas as pd

In [4]:
def get_id_range(start_id, end_id):
    start_num = int(start_id.split('-')[-1])
    end_num = int(end_id.split('-')[-1])
    return [f"generation-for-nlp-{i}" for i in range(start_num, end_num + 1)]

In [5]:
subject_ranges = {
    '수능 국어': {
        '2020년': 'generation-for-nlp-108~generation-for-nlp-142',
        '2021년': 'generation-for-nlp-48~generation-for-nlp-83',
        '2022년': 'generation-for-nlp-360~generation-for-nlp-398',
        '2023년': 'generation-for-nlp-0~generation-for-nlp-35',
        '2024년': 'generation-for-nlp-168~generation-for-nlp-209'
    },
    '수능 정치와 법': {
        '2020년': 'generation-for-nlp-217~generation-for-nlp-228',
        '2021년': 'generation-for-nlp-157~generation-for-nlp-167',
        '2022년': 'generation-for-nlp-339~generation-for-nlp-349',
        '2023년': 'generation-for-nlp-263~generation-for-nlp-276',
        '2024년': 'generation-for-nlp-36~generation-for-nlp-47'
    },
    '수능 사회문화': {
        '2020년': 'generation-for-nlp-210~generation-for-nlp-216',
        '2021년': 'generation-for-nlp-229~generation-for-nlp-238',
        '2022년': 'generation-for-nlp-84~generation-for-nlp-92',
        '2023년': 'generation-for-nlp-305~generation-for-nlp-315',
        '2024년': 'generation-for-nlp-316~generation-for-nlp-326'
    },
    '수능 생활과 윤리': {
        '2020년': 'generation-for-nlp-143~generation-for-nlp-156',
        '2021년': 'generation-for-nlp-399~generation-for-nlp-412',
        '2022년': 'generation-for-nlp-239~generation-for-nlp-250',
        '2023년': 'generation-for-nlp-251~generation-for-nlp-262',
        '2024년': 'generation-for-nlp-93~generation-for-nlp-107'
    },
    '수능 윤리와 사상': {
        '2020년': 'generation-for-nlp-327~generation-for-nlp-338',
        '2021년': 'generation-for-nlp-277~generation-for-nlp-289',
        '2022년': 'generation-for-nlp-290~generation-for-nlp-304',
        '2023년': 'generation-for-nlp-413~generation-for-nlp-424',
        '2024년': 'generation-for-nlp-350~generation-for-nlp-359'
    }
}

In [12]:
real_df = pd.read_csv('output_real.csv')
pred_df = pd.read_csv('ensemble_result.csv')

In [13]:
results = {}
subject_totals = {}
all_correct = 0
all_problems = 0

for subject, years in subject_ranges.items():
    results[subject] = {}
    total_correct = 0  # 과목별 총 정답 수
    total_problems = 0  # 과목별 총 문제 수
        
    for year, id_range in years.items():
        start_id, end_id = id_range.split('~')
        start_id = start_id.strip()
        end_id = end_id.strip()
            
        problem_ids = get_id_range(start_id, end_id)
            
        real_answers = real_df[real_df['id'].isin(problem_ids)]
        pred_answers = pred_df[pred_df['id'].isin(problem_ids)]
            
        comparison = pd.merge(real_answers, pred_answers, on='id', suffixes=('_real', '_pred'))
            
        total = len(problem_ids)
        correct = (comparison['answer_real'] == comparison['answer_pred']).sum()
            
        accuracy = (correct / total) * 100
        results[subject][year] = {
            'accuracy': accuracy,
            'correct': correct,
            'total': total
        }
        
        # 과목별 총계
        total_correct += correct
        total_problems += total
        
        all_correct += correct
        all_problems += total
    
    subject_totals[subject] = {
        'total_correct': total_correct,
        'total_problems': total_problems,
        'overall_accuracy': (total_correct / total_problems * 100)
    }

all_accuracy = (all_correct / all_problems * 100)

print("\n=== 결과 ===")
for subject, years in results.items():
    print(f"\n{subject}")
    for year, stats in years.items():
        print(f"  {year}: {stats['accuracy']:.1f}% ({stats['total']}개 중 {stats['correct']}개 정답)")

print("\n=== 과목별 총계 ===")
for subject, totals in subject_totals.items():
    print(f"{subject}: {totals['overall_accuracy']:.1f}% ({totals['total_problems']}개 중 {totals['total_correct']}개 정답)")
    
print("\n=== 과목별 총계 ===")
print(f"전체 정확도: {all_accuracy:.1f}% ({all_problems}개 중 {all_correct}개 정답)")
print(f"틀린 문제 수: {all_problems - all_correct}개")


=== 결과 ===

수능 국어
  2020년: 60.0% (35개 중 21개 정답)
  2021년: 50.0% (36개 중 18개 정답)
  2022년: 43.6% (39개 중 17개 정답)
  2023년: 66.7% (36개 중 24개 정답)
  2024년: 52.4% (42개 중 22개 정답)

수능 정치와 법
  2020년: 25.0% (12개 중 3개 정답)
  2021년: 18.2% (11개 중 2개 정답)
  2022년: 45.5% (11개 중 5개 정답)
  2023년: 42.9% (14개 중 6개 정답)
  2024년: 16.7% (12개 중 2개 정답)

수능 사회문화
  2020년: 57.1% (7개 중 4개 정답)
  2021년: 50.0% (10개 중 5개 정답)
  2022년: 22.2% (9개 중 2개 정답)
  2023년: 36.4% (11개 중 4개 정답)
  2024년: 27.3% (11개 중 3개 정답)

수능 생활과 윤리
  2020년: 78.6% (14개 중 11개 정답)
  2021년: 35.7% (14개 중 5개 정답)
  2022년: 41.7% (12개 중 5개 정답)
  2023년: 41.7% (12개 중 5개 정답)
  2024년: 66.7% (15개 중 10개 정답)

수능 윤리와 사상
  2020년: 75.0% (12개 중 9개 정답)
  2021년: 46.2% (13개 중 6개 정답)
  2022년: 46.7% (15개 중 7개 정답)
  2023년: 8.3% (12개 중 1개 정답)
  2024년: 40.0% (10개 중 4개 정답)

=== 과목별 총계 ===
수능 국어: 54.3% (188개 중 102개 정답)
수능 정치와 법: 30.0% (60개 중 18개 정답)
수능 사회문화: 37.5% (48개 중 18개 정답)
수능 생활과 윤리: 53.7% (67개 중 36개 정답)
수능 윤리와 사상: 43.5% (62개 중 27개 정답)

=== 과목별 총계 ===
전체 정확도: 47.3% (425개 중 20